In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.decomposition import PCA, SparsePCA
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed 
from tqdm import tqdm  
from tqdm_joblib import tqdm_joblib
from tqdm.notebook import tqdm
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error

pip install tqdm_joblib

folder_path = "/Users/naekyung/Desktop/A/00321804(0002)"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

required_features = ['name1', 'name2'] + [f'name{i:02}' for i in range(1, 15)]

results = {}

for idx, file in enumerate(csv_files, start=1):
    df = pd.read_csv(file)

    for column in df.columns:
        if column not in required_features:
            df.drop(columns=[column], inplace=True)

    dataframes = {}
    for i in range(1, 19):
        for j in range(1, 18):
            list_name = f'list_{i}_{j}'
            temp_list = df[(df['rack_no'] == i) & (df['module_no'] == j)].drop(columns=['rack_no', 'module_no']).values.tolist()
            if temp_list:
                columns = [f'{i}_{j}_{k+1}' for k in range(len(temp_list[0]))]
                temp_df = pd.DataFrame(temp_list, columns=columns)
                dataframes[list_name] = temp_df

    if dataframes:
        combined_df = pd.concat(dataframes.values(), axis=1)
        result_name = f'A_4_{idx}'
        results[result_name] = combined_df

for name, dataframe in results.items():
    print(f"DataFrame: {name}")
    print(dataframe)

# 모든 데이터프레임을 옆으로 병합
combined_results_df = pd.concat(results.values(), axis=1)

# 결과 확인
print("Combined DataFrame:")
print(combined_results_df)

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import RobustScaler
import joblib  # 모델 저장을 위해 사용

# 데이터 스케일링
scaler = RobustScaler()
scaled_combined_data = scaler.fit_transform(combined_results_df)
scaled_combined_df = pd.DataFrame(scaled_combined_data, columns=combined_results_df.columns)

# 데이터 전치 (231336*1440)
transposed_df = scaled_combined_df.T  

print("Scaled_combined_df.T:")
print(transposed_df)

# IPCA *****************************************************************************************************
n_components = 400 # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 주성분
batch_size = 1500

ipca = IncrementalPCA(n_components=n_components, batch_size=batch_size)
transformed_data = ipca.fit_transform(transposed_df)

# 변환된 데이터를 데이터프레임으로 저장
transformed_df = pd.DataFrame(transformed_data, columns=[f'PC{i+1}' for i in range(n_components)])
print("Transformed DataFrame:")
print(transformed_df)

# IPCA 모델 저장 (다른 데이터 복원에 사용하기 위해)
joblib.dump(ipca, 'ipca_model.pkl')
print("IPCA 모델이 저장되었습니다: ipca_model.pkl")

# 복원 ********************************************************************************************************
# 역변환을 통해 데이터 복원 (역스케일링 안 함)
reconstructed_data = ipca.inverse_transform(transformed_data)

# 원래 형식으로 복원 후 다시 전치
reconstructed_df = pd.DataFrame(reconstructed_data.T, columns=combined_results_df.columns)
print("Reconstructed DataFrame:")
print(reconstructed_df)

# MSE 계산 ****************************************************************************************************8

# 원본 데이터와 복원된 데이터 간 MSE 계산
mse = np.mean((scaled_combined_df.values - reconstructed_df.values) ** 2)
print("IPCA 복원 후 평균 제곱 오차(MSE):", mse)

import pandas as pd

# 경로 설정
file_path = "/Users/naekyung/Desktop/blind_test/A_2.csv"

# CSV 파일 읽기
blind_test_df = pd.read_csv(file_path)

# blind 데이터 전처리 하는 부분 (기밀 생략)

# 두 데이터프레임 로드
combined_df_1 = pd.read_csv("combined_bsc_fg_no_1.csv")  # bsc_fg_no=1 결과
combined_df_2 = pd.read_csv("combined_bsc_fg_no_2.csv")  # bsc_fg_no=2 결과

# 가로 병합
final_combined_df = pd.concat([combined_df_1, combined_df_2], axis=1)

# 결과 확인
print("가로 병합된 최종 데이터프레임:")
print(final_combined_df)

from sklearn.preprocessing import RobustScaler
import pandas as pd

# RobustScaler를 사용하여 스케일링
scaler = RobustScaler()
scaled_blind_data = scaler.fit_transform(final_combined_df)  # final_combined_df 인풋으로 받음
scaled_blind_df = pd.DataFrame(scaled_blind_data, columns=final_combined_df.columns)

# 데이터 전치
transposed_blind_df = pd.DataFrame(scaled_blind_df.T)

# 결과 확인
print("Transposed Blind Scaled DataFrame:")
print(transposed_blind_df)

# 정상 데이터로 학습된 IPCA 모델 로드
ipca = joblib.load('ipca_model.pkl')
print("IPCA 모델 로드 완료: ipca_model.pkl")

# 피처 개수 검증 및 조정
expected_features = ipca.components_.shape[1]  # IPCA 모델의 피처 개수
actual_features = transposed_blind_df.shape[1]

if actual_features < expected_features:
    # 부족한 피처를 채움
    for i in range(expected_features - actual_features):
        transposed_blind_df[f'missing_feature_{i}'] = 0
elif actual_features > expected_features:
    # 초과하는 피처를 제거
    transposed_blind_df = transposed_blind_df.iloc[:, :expected_features]

# 데이터 복원 (각 행에 대해 복원 수행)
reconstructed_data_list = []

for index in range(transposed_blind_df.shape[0]):
    row = transposed_blind_df.iloc[index:index+1, :].values
    transformed_row = ipca.transform(row)  # PCA 변환
    reconstructed_row = ipca.inverse_transform(transformed_row)  # PCA 복원
    reconstructed_data_list.append(reconstructed_row)

# 복원된 데이터를 데이터프레임으로 변환
reconstructed_blind_df = pd.DataFrame(
    np.vstack(reconstructed_data_list), 
    columns=transposed_blind_df.columns
)

# MSE 계산
mse = np.mean((transposed_blind_df.values - reconstructed_blind_df.values) ** 2)
print("블라인드 데이터 복원 후 평균 제곱 오차(MSE), 주성분 : 100 :", mse)